# Lineas de subte 
Despues de estudiar el mapa de Capital Federal nos dimos cuenta que hay una zona que esta cubierta por el subte y si nos alejamos de ella, hay una zona donde este medio de transporte no brinda servicio, en vez de separar el estudio por linea decidimos comparar todas las propiedades de Capital, siepre con su distancia menor a una linea de subte, a vr como influye esto con el precio

! [Imagen] (http://elpaisdigital.com.ar/media/contents/7fc7948a549c6dd37db99f38780bbb5c30cf8706.png)

In [1]:
import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config IPCompleter.greedy=True

#Data de properati
df = pd.read_csv('/home/matias/Escritorio/notebook/properati-AR-2017-08-01-properties-sell.csv')

#Filtramos solo las propiedades de Capital Federal
propCF = df[df['state_name']== 'Capital Federal']

propDepto = propCF[propCF['property_type'] == 'apartment']

propCapital = propDepto[['property_type','place_name','rooms','lat','lon','price']]

propCapital.dropna(inplace = True)
propCapital.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16903 entries, 162 to 186306
Data columns (total 6 columns):
property_type    16903 non-null object
place_name       16903 non-null object
rooms            16903 non-null float64
lat              16903 non-null float64
lon              16903 non-null float64
price            16903 non-null float64
dtypes: float64(4), object(2)
memory usage: 924.4+ KB


/home/matias/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [2]:
estaciones = pd.read_csv('/home/matias/Escritorio/notebook/estaciones-de-subte.csv')
estaciones.head()

estaciones.head(20)

,X,Y,ID,ESTACION,LINEA
0,-58.398928,-34.635750,1.0,CASEROS,H
1,-58.400970,-34.629376,2.0,INCLAN,H
2,-58.402323,-34.623092,3.0,HUMBERTO 1�,H
3,-58.404732,-34.615242,4.0,VENEZUELA,H
4,-58.406036,-34.608935,5.0,ONCE - 30 DE DICIEMBRE,H
5,-58.380574,-34.604245,6.0,9 DE JULIO,D
6,-58.397924,-34.599757,7.0,FACULTAD DE MEDICINA,D
7,-58.385142,-34.601587,8.0,TRIBUNALES - TEATRO COL�N,D
8,-58.407161,-34.591628,9.0,AG�ERO,D
9,-58.415955,-34.585156,10.0,R.SCALABRINI ORTIZ,D


In [3]:
i = 1
for index, row in estaciones.iterrows():
    lon = estaciones.loc[index,'X']
    lat = estaciones.loc[index,'Y']
    label = str(i)
    propCapital['lon' + label] = lon
    propCapital['lat' + label] = lat
    i = i+1
    


/home/matias/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/matias/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [4]:
propCapital.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16903 entries, 162 to 186306
Columns: 178 entries, property_type to lat86
dtypes: float64(176), object(2)
memory usage: 23.1+ MB


In [5]:
import math

def haversine(lat1, lon1, lat2, lon2):
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(np.sin(rad*dlat/2))**2 + np.cos(rad*lat1)*np.cos(rad*lat2)*(np.sin(rad*dlon/2))**2
    distancia=2*R*np.arcsin(np.sqrt(a))
    distM = distancia
    return distM

In [ ]:
for row in estaciones.iterrows():
    for i in range(1,87):
        label = str(i)
        lat1 = propCapital['lat'].astype(float)
        lon1 = propCapital['lon'].astype(float)
        lat2 = propCapital['lat'+ label].astype(float)
        lon2 = propCapital['lon'+ label].astype(float)
        dist = haversine(lat1,lon1,lat2,lon2)
        propCapital['dist' + label] = dist

In [ ]:
propCapital.head()

In [ ]:
columnas = 87
cols_to_use = ['dist'+str(i) for i in range(columnas)]
propCapital['min_dist'] =propCapital.loc[:,cols_to_use].min(axis=1)
propCapital.head()


In [ ]:
%matplotlib inline
propCapital1 = propCapital[propCapital['price']< 1000000]
propCapital2 = propCapital1[propCapital['min_dist']< 10]
propCapital2.plot.scatter('min_dist', 'price',alpha=0.25, figsize=(12,8))

Podemos concluir que efectivamente la distancia a las paradas de subte influye en los precios de las propiedades, si bien tenemos una concentracion grande de propiedades en el rango de 1 km de distnacia al subte podemos ver que a medida que nos alejamos mas la densidad de propiedas con precios altos decrece tremendamente. Con esto podemos llegar a dos conclusiones.
1. el precio de las propiedades disminuye a medida de que nos alejamos de cualquier linea de subte en un rango de 1 km aproximadamente
2. La concentracion de propiedades disminuye a medida de que nos alejamos de las paradas del subte, esto puede ser debido a que la presencia de estos medios de transporte favorece la urbanizacion